In [2]:
##############  Grads to Geotiff  Script   #################################
# Copyright Ryan L. Crumley, 2019
# ryanlcrumley@gmail.com

# Description: This script takes matfiles and writes a geotiff from each file. It uses numpy arrays and 
# GDAL to create the Geotiff.

# Import all of the python packages used in this workflow.
import scipy
import numpy as np
import matplotlib.pyplot as plt
import os, sys
from pylab import *
import pandas as pd
import numpy as np
import gdal
import osr
import scipy.io as sio
from os import listdir
import csv
print('All packages imported successfully')

# Define some things for use in the script.
year = '2017'
#year = '2018'

# Define the grid size in x and y values
nx = 2570
ny = 2480



All packages imported successfully


In [3]:
########### MAT_TO_ARRAY Function ########################## 
# This function reads in a matfile and turns it into a numpy array.

def mat_to_array(filename,nx,ny):
    
    # Load the mat contents to a numpy array using scipy file io
    mat_contents = sio.loadmat(filename)
    # Only load the variable that you care about
    numpy_data = mat_contents['swe_depth']
    # Return the results of the numpy array
    return numpy_data

print('Mat_to_Array is ready')

Mat_to_Array is ready


In [4]:
############# ARRAY_TO_GEOTIFF Function #######################

# This function takes the data from a single numpy array and turns it into a geotiff.
# The numpy arrays get used later.

def array_to_geotiff(filename,width,height,numpy_array):
    
    ######## USER INPUT SECTION ##################
    # Define some things about your model domain
    upper_left_x = 404791
    x_resolution = 30
    x_skew = 0
    upper_left_y = 1243170
    y_skew = 0
    y_resolution = 30
    #################################################
    
    # Reference the correct geotiff driver in GDAL
    driver = gdal.GetDriverByName("GTiff")
    
    # Begin to set the correct spatial reference to the data
    datasetSRS = osr.SpatialReference()
    
    ##############  USER INPUT SECTION   ############
    # Define the CRS using the EPSG code or your choice
    datasetSRS.ImportFromEPSG(3338)
    ##################################################
    
    # This creates the initial raster dataset. The 4th argument (1) is the number of bands in the input dataset.
    dataset = driver.Create(filename, width, height, 1, gdal.GDT_Float32)
    
    # Set the projection give the geotiff the correct spatial location as previously defined 
    dataset.SetGeoTransform((upper_left_x, x_resolution, x_skew, upper_left_y, y_skew, y_resolution))
    dataset.SetProjection(datasetSRS.ExportToWkt())
    
    # Create the output dataset from with one band
    outband = dataset.GetRasterBand(1)
    # This is the numpy array from the input dataset 
    outband.WriteArray(numpy_array)
    # This took me a bit to figure out, but there needs to be a Flush Cache call here in order for this to work,
    outband.FlushCache()  
    
    
print('Array_to_Geotiff is ready')

Array_to_Geotiff is ready


In [5]:
#singlefilename = 'woAssim_2017_calibrated_swed-slice.mat'
singlefilename = 'swed_timeslice.mat'

# Pull in the mat files from the folder one by one and change them to numpy arrays.
data_array = mat_to_array(singlefilename,nx,ny)

# Change the filename a bit.
shortname = singlefilename[0:-4]
output_filename = year+'_'+shortname+'.tif'
    
# Turn the numpy arrays into geotiffs and save in the geotiffs folder.
array_to_geotiff('geotiffs/'+output_filename,nx,ny,data_array)
    
# Print out some stuff
print('....working on file '+year+' '+shortname)

....working on file 2017 swed_timeslice
